In [3]:
def test_microphone():
    try:
        # Test microphone access with minimal duration
        duration = 0.1  # Very short duration test
        sample_rate = 44100
        recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
        sd.wait()
        print("✅ Microphone access successful!")
        return True
    except Exception as e:
        print("❌ Microphone access error!")
        print("\nTroubleshooting steps:")
        if sys.platform == 'darwin':  # macOS
            print("1. Go to System Settings > Privacy & Security > Microphone")
            print("2. Enable microphone access for PyCharm")
            print("3. Restart PyCharm and try again")
        elif sys.platform.startswith('win'):  # Windows
            print("1. Go to Settings > Privacy > Microphone")
            print("2. Turn on 'Allow apps to access your microphone'")
            print("3. Restart your Python environment (e.g., PyCharm, Anaconda)")
        elif sys.platform.startswith('linux'):  # Linux
            print("1. Ensure you have the necessary permissions to access audio devices")
            print("2. Check that your user is part of the 'audio' group")
            print("3. Restart your terminal or IDE and try again")
        else:
            print("1. Ensure your microphone is properly connected and has permissions")
        print(f"\nTechnical error details: {str(e)}")
        return False

In [4]:
import sounddevice as sd
print("Available audio devices:")
print(sd.query_devices())

Available audio devices:
   0 LG HDR DQHD, Core Audio (0 in, 2 out)
   1 Scarlett 18i8 USB, Core Audio (20 in, 8 out)
   2 Jabra SPEAK 510 USB, Core Audio (0 in, 2 out)
   3 Jabra SPEAK 510 USB, Core Audio (1 in, 0 out)
   4 Logitech BRIO, Core Audio (2 in, 0 out)
   5 BlackHole 16ch, Core Audio (16 in, 16 out)
   6 BlackHole 2ch, Core Audio (2 in, 2 out)
   7 External Headphones, Core Audio (0 in, 2 out)
   8 MacBook Pro Microphone, Core Audio (1 in, 0 out)
   9 MacBook Pro Speakers, Core Audio (0 in, 2 out)
  10 BoomAudio, Core Audio (6 in, 6 out)
  11 Microsoft Teams Audio, Core Audio (1 in, 1 out)
  12 ZoomAudioDevice, Core Audio (2 in, 2 out)
> 13 Blackhole/Scarlett Aggregate Device, Core Audio (22 in, 10 out)
< 14 Blackhole/Scarlett Out, Core Audio (0 in, 2 out)


In [5]:
def apply_gain(audio, gain_db):
    """Apply gain to audio signal"""
    return audio * (10 ** (gain_db / 20))

def apply_filter(audio, cutoff_freq):
    """Apply a simple lowpass filter"""
    from scipy import signal
    b, a = signal.butter(4, cutoff_freq / (SAMPLE_RATE/2))
    return signal.filtfilt(b, a, audio)

def process_audio(audio_input, gain_db=0, cutoff_freq=20000):
    """Main audio processing chain"""
    if audio_input is None:
        return None

    # Apply gain
    audio_with_gain = apply_gain(audio_input, gain_db)

    # Apply filter
    processed_audio = apply_filter(audio_with_gain, cutoff_freq)

    return processed_audio

#

In [8]:
import gradio as gr


def audio_interface():
    with gr.Blocks() as app:
        gr.Markdown("# Audio Signal Chain")

        with gr.Row():
            audio_input = gr.Audio(label="Input Audio")
            audio_output = gr.Audio(label="Processed Audio")

        with gr.Row():
            gain_slider = gr.Slider(
                minimum=-20,
                maximum=20,
                value=0,
                label="Gain (dB)"
            )
            cutoff_slider = gr.Slider(
                minimum=20,
                maximum=20000,
                value=20000,
                label="Low-Pass Filter Cutoff (Hz)"
            )

        audio_input.change(
            fn=process_audio,
            inputs=[audio_input, gain_slider, cutoff_slider],
            outputs=[audio_output]
        )

    return app


interface = audio_interface()

def audio_interface():
    with gr.Blocks() as app:
        gr.Markdown("# Audio Signal Chain")

        with gr.Row():
            audio_input = gr.Audio(
                sources=["microphone"],
                type="numpy",
                interactive=True,
                streaming=True,
                label="Audio Input (Focusrite Scarlett)"
            )
            audio_output = gr.Audio(
                label="Processed Output",
                type="numpy",
                autoplay=True
            )

        with gr.Row():
            gain_slider = gr.Slider(
                minimum=-20,
                maximum=20,
                value=0,
                step=0.1,
                label="Gain (dB)"
            )
            cutoff_slider = gr.Slider(
                minimum=20,
                maximum=20000,
                value=20000,
                step=10,
                label="Lowpass Filter Cutoff (Hz)"
            )

        audio_input.change(
            fn=process_audio,
            inputs=[audio_input, gain_slider, cutoff_slider],
            outputs=[audio_output]
        )

    return app

interface = audio_interface()

#

/Users/feamster/PycharmProjects/AudioSignalChain/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://abb4c333ca4a03bc27.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
